In [1]:
import numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import time
import tracemalloc
import linecache
import os
from scipy.spatial import distance
from sklearn.metrics.cluster import adjusted_rand_score
from pingouin import distance_corr
from scipy.stats import wasserstein_distance




tracemalloc.start()
start = time.time()

In [2]:
cell2loc = pd.read_csv("Cell2location.csv")
groundtruth = pd.read_csv("groundtruth_byspot.csv")
dwls = pd.read_csv("SpatialDWLS_seqFISH_10000.csv")
spotlight = pd.read_csv("SPOTlight_pred_cell.csv")
loc_data = "svz_cellcentroids.csv"
st_data = "svz_gene_expression_10000_genes.csv"

In [3]:
celltypes = ["olig", "astrocytes", "eNeuron", "endo_mural", "iNeuron", "microglia"]


In [4]:
svz_gene = pd.read_csv(st_data, header=0, index_col=0)
svz_gene = svz_gene.rename(columns=lambda x: x.upper())
# df_st should be Spots x Genes


svz_centroid = pd.read_csv(loc_data, header=0, index_col=0)
# df loc should be Spots x (x, y)

svz = svz_centroid.merge(svz_gene, on = "cellname", how="left")
first_col = svz.pop("cellname")
svz.insert(0, "cellname", first_col)
svz.drop(svz[svz["celltype"] == "Unannotated"].index, inplace = True)
svz.head()

,cellname,Cell ID,X,Y,Region,louvain,celltype,grid,spot,1700022A21RIK,...,TYRP1,UBQLNL,PTGDR,P2RY4,PRDM6,PDE6B,ZP1,DLX4,OPN1SW,PRAMEF12
0,0_54,54,267.56,245.42,Cortex,25,Oligodendrocytes,1_1,1,0,...,1,0,0,0,1,0,0,1,0,0
1,0_70,70,152.20,367.61,Cortex,6,Excitatory neuron,1_2,2,0,...,0,0,0,0,0,0,1,1,0,0
2,0_73,73,228.45,471.04,Cortex,9,Excitatory neuron,1_2,2,0,...,0,1,0,0,0,0,0,1,0,0
3,0_84,84,238.72,602.18,Cortex,6,Excitatory neuron,1_3,3,1,...,0,0,0,0,0,0,0,0,0,0
4,6_42,42,254.13,660.79,Choroid Plexus,1,Endothelial,1_4,4,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
svz["celltype"].value_counts()

celltype
Excitatory neuron    420
Endothelial          187
Astrocytes           130
Interneuron          105
Oligodendrocytes      41
Microglia             23
Name: count, dtype: int64

In [6]:
celltype_map = {"Excitatory neuron": "eNeuron", "Endothelial": "endo_mural", "Astrocytes": "astrocytes", 
                "Interneuron": "iNeuron", "Oligodendrocytes": "olig", "Microglia": "microgolia"}
svz["celltype"] = svz["celltype"].apply(lambda x: celltype_map[x])

In [7]:
def pcc(merged_df, gene_names, cell_type_name):

    """
    Calculates the Pearson correlation coefficient (PCC) between gene expression and cell type proportion.

    Args:
        merged_df: Pandas DataFrame containing gene expression data, cell type proportions, and other relevant columns ('X', 'Y', gene names, and cell type names).
        gene_names: A list of gene names (strings) for which to calculate the PCC.
        cell_type_name: The name of the cell type column (string) in merged_df.

    Returns:
        A Pandas Series containing the PCC values for each gene.  Returns an empty Series if gene_names is empty.
    """

    X = merged_df['X'].to_numpy()
    Y = merged_df['Y'].to_numpy()
    
    corr_list = []
    
    for gene_name in gene_names:
        gene_value = numpy.zeros(X.shape)
        gene_value += merged_df[gene_name].to_numpy()

        # cell_type_name = "olig"
        cell_type_prop = merged_df[cell_type_name].to_numpy()

        # Printing out info
        mean_gene_value = np.mean(gene_value)
        mean_cell_prop = np.mean(cell_type_prop)
        up = np.sum((gene_value - mean_gene_value) * (cell_type_prop - mean_cell_prop))
        down = np.sqrt(np.sum((gene_value - mean_gene_value) ** 2) * np.sum((cell_type_prop - mean_cell_prop) ** 2))
        pcc_corr = up/down
        corr_list.append(pcc_corr)
    return pd.Series(corr_list)



def create_celltype_corr_df(merged_df,gene_names, celltypes):

    """
    Creates a DataFrame of Pearson correlation coefficients between gene expression and multiple cell types.

    Args:
        merged_df: Pandas DataFrame containing gene expression data and cell type proportions.
        gene_names: A list of gene names (strings).
        celltypes: A list of cell type names (strings).

    Returns:
        A Pandas DataFrame where rows represent cell types, columns represent genes, and values are PCCs.
        Returns an empty DataFrame if gene_names or celltypes are empty.
    """

    results = []
    for cell_type in celltypes:
        result = pcc(merged_df, gene_names, cell_type)
        results.append(result)
    result_df = pd.DataFrame(results)
    result_df.columns = gene_names
    result_df.index = celltypes
    return result_df

def create_celltype_spearmanr_df(merged_df,gene_names, celltypes):

    """
    Creates a DataFrame of Pearson correlation coefficients between gene expression and multiple cell types.

    Args:
        merged_df: Pandas DataFrame containing gene expression data and cell type proportions.
        gene_names: A list of gene names (strings).
        celltypes: A list of cell type names (strings).

    Returns:
        A Pandas DataFrame where rows represent cell types, columns represent genes, and values are PCCs.
        Returns an empty DataFrame if gene_names or celltypes are empty.
    """

    results = []
    for cell_type in celltypes:
        result = spearman(merged_df, gene_names, cell_type)
        results.append(result)
    result_df = pd.DataFrame(results)
    result_df.columns = gene_names
    result_df.index = celltypes
    return result_df



def calculate_rmse(groundtruth_df, method_name, method_df, celltypes):

    """
    Calculates the Root Mean Squared Error (RMSE) between ground truth and a method's cell type proportions.

    Args:
        groundtruth_df: Pandas DataFrame containing ground truth cell type proportions. Must have a 'spot' column.
        method_name: String, the name of the method being evaluated (used for column suffix).
        method_df: Pandas DataFrame containing the method's cell type proportions. Must have a 'spot' column.
        celltypes: List of strings, names of cell types.

    Returns:
        The RMSE value (float). 

    """

    method_suffix = "_" + method_name

    merged_df = pd.merge(groundtruth_df, method_df, on = "spot", how = "inner", suffixes = ('_ground', method_suffix))

    rmse = 0
    rmse_sum = 0
    for celltype in celltypes:
        ground_col = celltype + "_ground"
        method_col = celltype + method_suffix
        prop_sum = sum(merged_df[method_col])
        for _, row in merged_df.iterrows():
            rmse_sum += (row[method_col] - row[ground_col])**2
        rmse += rmse_sum/prop_sum
    rmse = np.sqrt(rmse_sum / len(celltypes))
    return rmse

def calculate_jsd_per_type(groundtruth_df, method_name, method_df, celltypes):

    """
    Calculates the Jensen-Shannon Divergence (JSD) between ground truth and a method's cell type distributions for each cell type.

    Args:
        groundtruth_df: Pandas DataFrame with ground truth cell type proportions. Must have a 'spot' column.
        method_name: String, name of the method.
        method_df: Pandas DataFrame with method's cell type proportions. Must have a 'spot' column.
        celltypes: List of strings, cell type names.

    Returns:
        A list of JSD values, one for each cell type. 
    """

    method_suffix = "_" + method_name
    merged_df = pd.merge(groundtruth_df, method_df, on = "spot", how = "inner", suffixes = ('_ground', method_suffix))

    jsd_list = []
    for celltype in celltypes:
        ground_col = celltype + "_ground"
        method_col = celltype + method_suffix
        ground_distributiion = merged_df[ground_col]
        method_distribution = merged_df[method_col]
        jsd = distance.jensenshannon(ground_distributiion, method_distribution)
        jsd_list.append(jsd)
    return jsd_list

def calculate_wass_dist_per_type(groundtruth_df, method_name, method_df, celltypes):
    method_suffix = "_" + method_name
    merged_df = pd.merge(groundtruth_df, method_df, on = "spot", how = "inner", suffixes = ('_ground', method_suffix))

    wass_list = []
    for celltype in celltypes:
        ground_col = celltype + "_ground"
        method_col = celltype + method_suffix
        ground_distributiion = merged_df[ground_col]
        method_distribution = merged_df[method_col]
        wass = wasserstein_distance(ground_distributiion, method_distribution)
        wass_list.append(wass)
    return wass_list


def spearman(merged_df, gene_names, cell_type_name):
    X = merged_df['X'].to_numpy()
    Y = merged_df['Y'].to_numpy()
    # genes = svz_gene.columns
    # for celltype in celltypes
    #     for gene in genes:
    # gene_name = ["CLDN2"]
    corr_list = []
    
    for gene_name in gene_names:
        gene_value = numpy.zeros(X.shape)
        gene_value += merged_df[gene_name].to_numpy()

        # cell_type_name = "olig"
        cell_type_prop = merged_df[cell_type_name].to_numpy()

        # Printing out info
        spearmanr_corr, p_value = stats.spearmanr(gene_value, cell_type_prop)
        corr_list.append(spearmanr_corr)
    return pd.Series(corr_list)

GroundTruth


In [8]:
print(groundtruth.shape)
groundtruth.rename(columns={"Unnamed: 0": "spot", 
                            "Excitatory neuron": "eNeuron", 
                            "Endothelial": "endo_mural", 
                            "Astrocytes": "astrocytes",
                            "Interneuron": "iNeuron", 
                            "Oligodendrocytes": "olig", 
                            "Microglia": "microglia"}, inplace=True)
svz_ground = pd.merge(groundtruth, svz, on="spot", how="inner")
svz_ground.head()

(71, 7)


,spot,eNeuron,endo_mural,astrocytes,iNeuron,olig,microglia,cellname,Cell ID,X,...,TYRP1,UBQLNL,PTGDR,P2RY4,PRDM6,PDE6B,ZP1,DLX4,OPN1SW,PRAMEF12
0,1,0.000000,0.000000,0.0,0.0,1.0,0.0,0_54,54,267.56,...,1,0,0,0,1,0,0,1,0,0
1,2,1.000000,0.000000,0.0,0.0,0.0,0.0,0_70,70,152.20,...,0,0,0,0,0,0,1,1,0,0
2,2,1.000000,0.000000,0.0,0.0,0.0,0.0,0_73,73,228.45,...,0,1,0,0,0,0,0,1,0,0
3,3,1.000000,0.000000,0.0,0.0,0.0,0.0,0_84,84,238.72,...,0,0,0,0,0,0,0,0,0,0
4,4,0.333333,0.666667,0.0,0.0,0.0,0.0,6_42,42,254.13,...,0,0,0,0,0,1,0,0,0,0


In [9]:
svz_ground["celltype"].value_counts()

celltype
eNeuron       317
endo_mural    137
astrocytes    103
iNeuron        80
olig           35
microgolia     22
Name: count, dtype: int64

In [10]:
svz_ground_pcc = create_celltype_corr_df(svz_ground, svz_gene.columns, celltypes)
svz_ground_spearman = create_celltype_spearmanr_df(svz_ground, svz_gene.columns, celltypes)

In [11]:
svz_ground_spearman.to_csv("SPEARMANR/ground_spearmanr.csv")
svz_ground_pcc.to_csv("PCC/ground_pcc.csv")


Cell2Location

In [12]:
pred_data = "Cell2location.csv"
cell2loc = pd.read_csv(pred_data)
cell2loc.rename(columns={'Unnamed: 0': "spot", 'meanscell_abundance_w_sf_Olig': "olig", 
                        'meanscell_abundance_w_sf_astrocytes': "astrocytes",
                        'meanscell_abundance_w_sf_eNeuron': "eNeuron",
                        'meanscell_abundance_w_sf_endo_mural': "endo_mural",
                        'meanscell_abundance_w_sf_iNeuron': "iNeuron",
                        'meanscell_abundance_w_sf_microglia': "microglia"}, inplace=True)
svz_cell2loc = pd.merge(cell2loc, svz, left_on="spot", right_on="spot", how="inner")
svz_cell2loc.head()

,spot,olig,astrocytes,eNeuron,endo_mural,iNeuron,microglia,cellname,Cell ID,X,...,TYRP1,UBQLNL,PTGDR,P2RY4,PRDM6,PDE6B,ZP1,DLX4,OPN1SW,PRAMEF12
0,1,0.042227,0.260268,0.007680,0.303891,0.133484,0.252450,0_54,54,267.56,...,1,0,0,0,1,0,0,1,0,0
1,2,0.045253,0.398171,0.016932,0.191088,0.059565,0.288991,0_70,70,152.20,...,0,0,0,0,0,0,1,1,0,0
2,2,0.045253,0.398171,0.016932,0.191088,0.059565,0.288991,0_73,73,228.45,...,0,1,0,0,0,0,0,1,0,0
3,3,0.031904,0.322007,0.020697,0.118548,0.155338,0.351507,0_84,84,238.72,...,0,0,0,0,0,0,0,0,0,0
4,7,0.032451,0.098628,0.171488,0.227526,0.073593,0.396313,0_33,33,103.12,...,1,3,0,0,0,0,0,2,0,0


In [13]:
svz_cell2loc["celltype"].value_counts()

celltype
eNeuron       259
endo_mural    117
astrocytes     72
iNeuron        66
olig           23
microgolia     12
Name: count, dtype: int64

In [14]:
svz_cell2loc_pcc = create_celltype_corr_df(svz_cell2loc, svz_gene.columns, celltypes)
svz_cell2loc_spearman = create_celltype_spearmanr_df(svz_cell2loc, svz_gene.columns, celltypes)
cell2loc_rmse = calculate_rmse(groundtruth, "cell2loc", cell2loc, celltypes)
cell2loc_jsd_lis = calculate_jsd_per_type(groundtruth, "cell2loc", cell2loc, celltypes)
cell2loc_wass_lis = calculate_wass_dist_per_type(groundtruth, "cell2loc", cell2loc, celltypes)

In [15]:
svz_cell2loc_spearman.to_csv("SPEARMANR/cell2loc_spearmanr.csv")
svz_cell2loc_pcc.to_csv("PCC/cell2loc_pcc.csv")

In [16]:
# def calculate_ari_per_type(groundtruth_df, method_name, method_df, celltypes):
#     method_suffix = "_" + method_name
#     merged_df = pd.merge(groundtruth_df, method_df, on = "spot", how = "inner", suffixes = ('_ground', method_suffix))
#     ari_list = []
#     for celltype in celltypes:
#         ground_col = celltype + "_ground"
#         method_col = celltype + method_suffix
#         ground_distributiion = merged_df[ground_col]
#         method_distribution = merged_df[method_col]
#         ari = silhouette_score(ground_distributiion, method_distribution)
#         ari_list.append(ari)
#     return ari_list
# calculate_ari_per_type(groundtruth, "cell2loc", cell2loc, celltypes)

DWLS

In [17]:
print(dwls.shape)
dwls.rename(columns = {'Unnamed: 0': "spot", 'cell_ID': "cell_ID", 
                       'Olig': "olig", 'microglia': "microglia", 
                       'endo_mural': "endo_mural", 
                       'astrocytes': "astrocytes", 
                       'iNeuron': "iNeuron", 'eNeuron': "eNeuron"}, inplace = True)
svz_dwls = pd.merge(dwls, svz, on="spot", how="inner")
svz_dwls.head()

(71, 8)


,spot,cell_ID,olig,microglia,endo_mural,astrocytes,iNeuron,eNeuron,cellname,Cell ID,...,TYRP1,UBQLNL,PTGDR,P2RY4,PRDM6,PDE6B,ZP1,DLX4,OPN1SW,PRAMEF12
0,1,0,0.0,0.0,0.913625,0.0,0.086375,0.000000,0_54,54,...,1,0,0,0,1,0,0,1,0,0
1,2,1,0.0,0.0,0.000000,0.0,0.000000,1.000000,0_70,70,...,0,0,0,0,0,0,1,1,0,0
2,2,1,0.0,0.0,0.000000,0.0,0.000000,1.000000,0_73,73,...,0,1,0,0,0,0,0,1,0,0
3,3,2,0.0,0.0,0.000000,0.0,0.289810,0.710190,0_84,84,...,0,0,0,0,0,0,0,0,0,0
4,4,3,0.0,0.0,0.000000,0.0,0.121941,0.878059,6_42,42,...,0,0,0,0,0,1,0,0,0,0


In [18]:
svz_cell2loc["celltype"].value_counts()

celltype
eNeuron       259
endo_mural    117
astrocytes     72
iNeuron        66
olig           23
microgolia     12
Name: count, dtype: int64

In [19]:
svz_dwls_pcc = create_celltype_corr_df(svz_dwls, svz_gene.columns, celltypes)
svz_dwls_spearman = create_celltype_spearmanr_df(svz_dwls, svz_gene.columns, celltypes)
dwls_rmse = calculate_rmse(groundtruth, "dwls", dwls, celltypes)
dwls_jsd_lis = calculate_jsd_per_type(groundtruth, "dwls", dwls, celltypes)
dwls_wass_lis = calculate_wass_dist_per_type(groundtruth, "dwls", dwls, celltypes)

In [20]:
svz_dwls_spearman.to_csv("SPEARMANR/dwls_spearmanr.csv")
svz_dwls_pcc.to_csv("PCC/dwls_pcc.csv")

Spotlight

In [21]:
print(spotlight.shape)
spotlight.rename(columns={'Unnamed: 0': "spot",
                          'astrocytes': "astrocytes", 
                          'endo_mural': "endo_mural", 
                          'eNeuron': "eNeuron",
                          'iNeuron': "iNeuron",
                          'microglia': "microglia", 
                          'Olig': "olig"}, inplace=True)
svz_spotlight = pd.merge(spotlight, svz, on="spot", how="inner")
svz_spotlight.head()

(71, 7)


,spot,astrocytes,endo_mural,eNeuron,iNeuron,microglia,olig,cellname,Cell ID,X,...,TYRP1,UBQLNL,PTGDR,P2RY4,PRDM6,PDE6B,ZP1,DLX4,OPN1SW,PRAMEF12
0,1,0.0,0.0,1,0.0,0,0.0,0_54,54,267.56,...,1,0,0,0,1,0,0,1,0,0
1,2,0.0,0.0,1,0.0,0,0.0,0_70,70,152.20,...,0,0,0,0,0,0,1,1,0,0
2,2,0.0,0.0,1,0.0,0,0.0,0_73,73,228.45,...,0,1,0,0,0,0,0,1,0,0
3,3,0.0,0.0,1,0.0,0,0.0,0_84,84,238.72,...,0,0,0,0,0,0,0,0,0,0
4,7,0.0,0.0,0,0.0,1,0.0,0_33,33,103.12,...,1,3,0,0,0,0,0,2,0,0


In [22]:
svz_spotlight["celltype"].value_counts()

celltype
eNeuron       259
endo_mural    117
astrocytes     72
iNeuron        66
olig           23
microgolia     12
Name: count, dtype: int64

In [23]:
svz_spotlight_pcc = create_celltype_corr_df(svz_spotlight, svz_gene.columns, celltypes)
svz_spotlight_spearman = create_celltype_spearmanr_df(svz_spotlight, svz_gene.columns, celltypes)
spotlight_rmse = calculate_rmse(groundtruth, "spotlight", spotlight, celltypes)
spotlight_jsd_lis = calculate_jsd_per_type(groundtruth, "spotlight", spotlight, celltypes)
spotlight_wass_lis = calculate_wass_dist_per_type(groundtruth, "spotlight", spotlight, celltypes)

In [24]:
svz_spotlight_spearman.to_csv("SPEARMANR/spotlight_spearmanr.csv")
svz_spotlight_pcc.to_csv("PCC/spotlight_pcc.csv")

Gather metric information of each method

In [25]:
rmse_list = [cell2loc_rmse, dwls_rmse, spotlight_rmse]
jsd_list = [cell2loc_jsd_lis, dwls_jsd_lis, spotlight_jsd_lis]
wass_list = [cell2loc_wass_lis, dwls_wass_lis, spotlight_wass_lis]
metric_df = pd.concat([pd.DataFrame(jsd_list), pd.DataFrame(wass_list)], axis = 1)
metric_df.columns = [celltype +"_jsd" for celltype in celltypes] + [celltype +"_wass" for celltype in celltypes]
metric_df.index = ["cell2loc", "dwls", "spotlight"]
metric_df["rmse"] = rmse_list
metric_df

,olig_jsd,astrocytes_jsd,eNeuron_jsd,endo_mural_jsd,iNeuron_jsd,microglia_jsd,olig_wass,astrocytes_wass,eNeuron_wass,endo_mural_wass,iNeuron_wass,microglia_wass,rmse
cell2loc,0.642640,0.454202,0.235941,0.432420,0.447934,0.641911,0.081224,0.075004,0.298552,0.092926,0.071419,0.328362,1.577324
dwls,0.609095,0.709484,0.513336,0.593045,0.692640,0.735585,0.113365,0.060235,0.287125,0.094522,0.038777,0.197648,2.705814
spotlight,0.798043,0.664281,0.690439,0.713957,0.672901,0.676684,0.063706,0.105555,0.356212,0.136025,0.081220,0.607210,2.780028


In [26]:
metric_df.to_csv("metric.csv")

In [27]:
# Save timestamp
end = time.time()

print(end - start)

439.2907803058624


In [28]:
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))


snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

Top 3 lines
#1: internals\managers.py:2299: 193139.5 KiB
    new_values = new_values[argsort]
#2: internals\blocks.py:796: 71537.8 KiB
    values = values.copy()
#3: array_algos\take.py:157: 70846.4 KiB
    out = np.empty(out_shape, dtype=dtype)
1172 other: 81420.0 KiB
Total allocated size: 416943.7 KiB
